In [1]:
import os
import torch
import numpy as np
from lib.network.rtpose_vgg import get_model

Below cells were copied from the picture demo

In [2]:
model = get_model('vgg19')     
model.load_state_dict(torch.load('pose_model.pth'))
model.float()
model.eval()

Bulding VGG19


rtpose_model(
  (model0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1

In [3]:
dummy_input = torch.randn(1, 3,427, 640)
torch_output = model(dummy_input)

In [4]:
torch.onnx.export(model, dummy_input, 'onnx_pose_model.onnx')

In [14]:
import torchsummary

In [19]:
torchsummary.summary(model, input_size=(3,427,640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 427, 640]           1,792
              ReLU-2         [-1, 64, 427, 640]               0
            Conv2d-3         [-1, 64, 427, 640]          36,928
              ReLU-4         [-1, 64, 427, 640]               0
         MaxPool2d-5         [-1, 64, 213, 320]               0
            Conv2d-6        [-1, 128, 213, 320]          73,856
              ReLU-7        [-1, 128, 213, 320]               0
            Conv2d-8        [-1, 128, 213, 320]         147,584
              ReLU-9        [-1, 128, 213, 320]               0
        MaxPool2d-10        [-1, 128, 106, 160]               0
           Conv2d-11        [-1, 256, 106, 160]         295,168
             ReLU-12        [-1, 256, 106, 160]               0
           Conv2d-13        [-1, 256, 106, 160]         590,080
             ReLU-14        [-1, 256, 1

In [17]:
dummy_input.shape

torch.Size([1, 3, 427, 640])

In [23]:
torch_output[0][0].shape

torch.Size([1, 38, 53, 80])

In [24]:
torch_output[0][1].shape

torch.Size([1, 19, 53, 80])

## Read ONNX model

In [6]:
import onnx

# Load the ONNX model
onnx_file = "onnx_pose_model.onnx"
model_onnx = onnx.load(onnx_file)

# Check that the IR is well formed
onnx.checker.check_model(model_onnx)

# Print a human readable representation of the graph
# onnx.helper.printable_graph(model_onnx.graph)

In [7]:
# ...continuing from above
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_file)

onnx_output = ort_session.run(None, {ort_session.get_inputs()[0].name: dummy_input.numpy()})
len(onnx_output)

14

In [8]:
np.testing.assert_allclose(torch_output[0][0].detach().numpy(), onnx_output[0], rtol=1e-09, atol=1e-05)
np.testing.assert_allclose(torch_output[0][1].detach().numpy(), onnx_output[1], rtol=1e-09, atol=1e-06)
np.testing.assert_allclose(torch_output[1][0].detach().numpy(), onnx_output[2], rtol=1e-09, atol=1e-05)

In [9]:
print(model_onnx.graph.output)

[name: "351"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 38
      }
      dim {
        dim_value: 53
      }
      dim {
        dim_value: 80
      }
    }
  }
}
, name: "364"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 19
      }
      dim {
        dim_value: 53
      }
      dim {
        dim_value: 80
      }
    }
  }
}
, name: "220"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 38
      }
      dim {
        dim_value: 53
      }
      dim {
        dim_value: 80
      }
    }
  }
}
, name: "229"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 19
      }
      dim {
        dim_value: 53
      }
      dim {
        dim_value: 80
      }
    }
  }
}
, name: "243"
type {


## ONNX to OpenVINO

In [10]:
cmd = 'python "C:\\Program Files (x86)\\IntelSWTools\\openvino\\deployment_tools\\model_optimizer\\mo_onnx.py" \
--input_model ' + onnx_file + ' --output_dir vino_models'

In [11]:
print(cmd)

python "C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_onnx.py" --input_model onnx_pose_model.onnx --output_dir vino_models


In [12]:
os.system(cmd)

0

## OpenVINO Inference

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  3 02:26:35 2020
@author: ADubey4

Note:       If you have not set the paths permanetely in environment.
            Then run this file on command promopt, preceded by below two commands
code Line:  cd C:\Program Files (x86)\IntelSWTools\openvino\bin\
code Line:  setupvars.bat
            then without closing the same command prompt, run this file
code Line:  python regression_pyTorch_infer_openVino.py
"""

from openvino.inference_engine import IENetwork, IEPlugin, IECore

plugin_dir = None
model_xml = r'vino_models\onnx_pose_model.xml'
model_bin = r'vino_models\onnx_pose_model.bin'

# plugin = IEPlugin("CPU", plugin_dirs=plugin_dir)
ie = IECore()
# versions = ie.get_versions("CPU")
# Read IR
net = IENetwork(model=model_xml, weights=model_bin)
# check net.inputs.keys(), net.outputs
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
# exec_net = plugin.load(network=net)
exec_net = ie.load_network(network=net, device_name="CPU")
del net

# Run inference
# x = np.random.random((1,3, 427, 640))
x = dummy_input.numpy()
res = exec_net.infer(inputs={input_blob: x})

ImportError: DLL load failed: The specified module could not be found.

In [20]:
np.testing.assert_allclose(torch_output[0][0].detach().numpy(), res['351'], rtol=1e-09, atol=1e-05)
np.testing.assert_allclose(torch_output[0][1].detach().numpy(), res['364'], rtol=1e-09, atol=1e-06)

NameError: name 'torch_output' is not defined